In [3]:
!pip install pyarrow

     |████████████████████████████████| 40.8 MB 26.9 MB/s eta 0:00:01


In [1]:
!python -V

Python 3.9.19


In [2]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

# Q1. Install MLflow

In [3]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment1")


<Experiment: artifact_location='/workspaces/MLOPS-Course/02-Experiment-Tracking/mlruns/2', creation_time=1716829170043, experiment_id='2', last_update_time=1716829170043, lifecycle_stage='active', name='nyc-taxi-experiment1', tags={}>

In [4]:
mlflow.__version__

'2.13.0'

# Q2. Downloading the data

We'll use the Green Taxi Trip Records dataset to predict the duration of each trip.

Download the data for January, February and March 2023 in parquet format from https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

Use the script preprocess_data.py located in the folder homework to preprocess the data.

The script will:

load the data from the folder <TAXI_DATA_FOLDER> (the folder where you have downloaded the data),
fit a DictVectorizer on the training set (January 2023 data),
save the preprocessed datasets and the DictVectorizer to disk.

Your task is to download the datasets and then execute this command:

python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output

In [30]:
# running preprocess script
!python preprocess_data.py --raw_data_path "/workspaces/MLOPS-Course/02-Experiment-Tracking/data" --dest_path ./output

In [31]:
# checking the files in output folder
!ls output/

dv.pkl	test.pkl  train.pkl  val.pkl


After running the pre-process script, now we have 4 files in output folder.

# Q3. Train a model with autolog

We will train a RandomForestRegressor (from Scikit-Learn) on the taxi dataset.

We have prepared the training script train.py for this exercise, which can be also found in the folder homework.

The script will:

load the datasets produced by the previous step,
train the model on the training set,
calculate the RMSE score on the validation set.
Your task is to modify the script to enable autologging with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked.

Tip 1: don't forget to wrap the training code with a with mlflow.start_run(): statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

In [13]:
# model training and experiment tracking through mlflow
!python train.py

2024/05/27 18:19:05 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/05/27 18:19:06 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/05/27 18:19:08 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/27 18:19:24 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/codespace/anaconda3/envs/experiment-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils."


In [14]:
# specify the experiment name or ID where your runs were logged
experiment_name = "nyc-taxi-experiment1"

# get the experiment ID
experiment = mlflow.get_experiment_by_name(experiment_name)
experiment_id = experiment.experiment_id

# search for runs in the experiment
runs = mlflow.search_runs(experiment_ids=[experiment_id])

# iterate through the runs and retrieve min_samples_split parameter
for run in runs.iterrows():
    run_id = run[1]['run_id']
    params = mlflow.get_run(run_id).data.params
    if 'min_samples_split' in params:
        min_samples_split = params['min_samples_split']
        print(f"Run ID: {run_id}, min_samples_split: {min_samples_split}")

Run ID: 852cd2ac0dbf42de824286d491467abc, min_samples_split: 2
Run ID: 6cdbab30c1e04c5a8d0359a923e1c803, min_samples_split: 2
Run ID: 211afe9a1a73420b97446e91ff221dba, min_samples_split: 2


So, value of the min_samples_split parameter is 2.

# Q4. Launch the tracking server locally

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry.

Your task is to:

launch the tracking server on your local machine,
select a SQLite db for the backend store and a folder called artifacts for the artifacts store.
You should keep the tracking server running to work on the next two exercises that use the server.

In addition to backend-store-uri, what else do you need to pass to properly configure the server?

In [18]:
# launch the tracking server
!mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./artifacts  --port 5002

# in addition to sqlite db, we need default-artifact-root

[2024-05-27 18:49:14 +0000] [153937] [INFO] Starting gunicorn 22.0.0
[2024-05-27 18:49:14 +0000] [153937] [INFO] Listening at: http://127.0.0.1:5002 (153937)
[2024-05-27 18:49:14 +0000] [153937] [INFO] Using worker: sync
[2024-05-27 18:49:14 +0000] [153938] [INFO] Booting worker with pid: 153938
[2024-05-27 18:49:14 +0000] [153939] [INFO] Booting worker with pid: 153939
[2024-05-27 18:49:14 +0000] [153940] [INFO] Booting worker with pid: 153940
[2024-05-27 18:49:14 +0000] [153941] [INFO] Booting worker with pid: 153941
^C
[2024-05-27 19:15:20 +0000] [153937] [INFO] Handling signal: int
[2024-05-27 19:15:20 +0000] [153938] [INFO] Worker exiting (pid: 153938)
[2024-05-27 19:15:20 +0000] [153941] [INFO] Worker exiting (pid: 153941)
[2024-05-27 19:15:20 +0000] [153940] [INFO] Worker exiting (pid: 153940)
[2024-05-27 19:15:20 +0000] [153939] [INFO] Worker exiting (pid: 153939)


# Q5. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the RandomForestRegressor using hyperopt. We have prepared the script hpo.py for this exercise.

Your task is to modify the script hpo.py and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the objective function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called random-forest-hyperopt to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

the list of hyperparameters that are passed to the objective function during the optimization,
the RMSE obtained on the validation set (February 2023 data).
What's the best validation RMSE that you got?

In [19]:
!python hpo.py

2024/05/27 19:15:30 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
  0%|                                    | 0/15 [00:00<?, ?trial/s, best loss=?]/home/codespace/anaconda3/envs/experiment-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

  7%|▋          | 1/15 [00:08<01:52,  8.07s/trial, best loss: 5.370086069268862]/home/codespace/anaconda3/envs/experiment-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

 13%|█▍         | 2/15 [00:08<00:46,  3.61s/trial, best loss: 5.370086069268862]/home/codespac

So, the best validation RMSE is 5.335

# Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. In this exercise, you'll promote the best model to the model registry. We have prepared a script called register_model.py, which will check the results from the previous step and select the top 5 runs. After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called random-forest-best-models.

Your task is to update the script register_model.py so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method search_runs from the MlflowClient to get the model with the lowest RMSE,

Tip 2: to register the model you can use the method mlflow.register_model and you will need to pass the right model_uri in the form of a string that looks like this: "runs:/<RUN_ID>/model", and the name of the model (make sure to choose a good one!).

In [23]:
!python register_model.py

/home/codespace/anaconda3/envs/experiment-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/codespace/anaconda3/envs/experiment-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/codespace/anaconda3/envs/experiment-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/codespace/anaconda3/envs/experiment-tracking-env/lib/python3.9/site-packages/sklearn/

In [24]:
from mlflow.tracking import MlflowClient

MODEL_NAME = "random_forest_best_model"
run_id = "fcaf25728d114ffea979aa9a8ed47565"

# Initialize the MLflow client
client = MlflowClient()


# Fetch the logged metrics for the run
run = client.get_run(run_id)
metrics = run.data.metrics

# Print the test RMSE
test_rmse = metrics.get("test_rmse")
print(f"Test RMSE of the best model: {test_rmse}")


Test RMSE of the best model: 5.567408012462019
